# qBraid Quantum Jobs: Amazon Braket Environment

In [1]:
# list installed environments
!qbraid envs list

Installed environments:
qsharp
default
amazon_braket
qbraid_sdk
Use `qbraid envs <environment> -h` to see available commands.


In [2]:
# enable qBraid Quantum Jobs
!qbraid jobs enable -n amazon_braket

Successfully enabled qBraid Quantum Jobs in the amazon_braket environment.
Every AWS job you run will now be submitted through the qBraid API, so no access keys are necessary. 

To disable, run: `qbraid jobs disable -n amazon_braket`


In [3]:
# check qBraid credits
!qbraid jobs get-credits

You have 1000 remaining qBraid credits.


In [4]:
import boto3
from braket.aws import AwsDevice, AwsQuantumTask 
from braket.circuits import Circuit

# create the Amazon Braket circuit
bell = Circuit().h(0).cnot(0, 1)

print(bell)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


In [5]:
# dummy variable for the name of the bucket
my_bucket = "MYBUCKET"
# dummy variable for the name of the folder in the bucket
my_prefix = "MYPREFIX"
# qBraid Quantum Jobs API takes care of this
s3_folder = (my_bucket, my_prefix)

In [6]:
# choose the cloud-based on-demand simulator to run your circuit
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

In [7]:
# execute the circuit
task = device.run(bell, s3_folder, shots=100)

In [8]:
# display the results
print(task.result().measurement_counts)

Counter({'11': 51, '00': 49})


In [9]:
# list submitted jobs
!qbraid jobs list

Mon Sep 12 2022 20:29:38 GMT+0000 (Coordinated Universal Time) arn:aws:braket:::device/quantum-simulator/amazon/sv1 arn:aws:braket:us-east-1:592242689881:quantum-task/8a470bc6-d79c-4820-83ec-dbe20273145f



In [10]:
# check remaining qBraid credits
!qbraid jobs get-credits

You have 999.94 remaining qBraid credits.


In [11]:
client = boto3.client('braket')

In [12]:
# retrieve jobs from braket boto3
boto_data = client.search_quantum_tasks(
    filters = [
        {
            'name': 'deviceArn',
            'operator': 'EQUAL',
            'values': [
                'arn:aws:braket:::device/quantum-simulator/amazon/sv1',
            ]
        }
    ],
)

# get data on most recent job
job_data = boto_data['quantumTasks'][0]

for field in job_data:
    print(field)

createdAt
deviceArn
endedAt
outputS3Bucket
outputS3Directory
quantumTaskArn
shots
status
tags


In [13]:
job_data['outputS3Bucket']

'amazon-braket-qbraid-jobs'

In [14]:
# disable qBraid Quantum Jobs
!qbraid jobs disable -n amazon_braket

Disable successful. You are now submitting quantum jobs with your own AWS credentials.

To re-enable, run: `qbraid jobs enable -n amazon_braket`
